In [2]:
import os
import cv2
import random
import pickle
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [3]:
Image= cv2.readimage('Data_Kurangi_Filter\Testing\diabetic_images\Diabetes_1.png')

AttributeError: module 'cv2' has no attribute 'readimage'

In [584]:
train_dir = os.path.join('Data_Kurangi_Filter/Training')
testing_dir = os.path.join('Data_Kurangi_Filter/Testing')

In [612]:
# Fungsi untuk melakukan augmentasi pada gambar
def augment_image(image):
    # Flip horizontal
    if random.random() < 0.5:
        image = np.flip(image, axis=1)

    # Flip vertical
    if random.random() < 0.5:
        image = np.flip(image, axis=0)

    # Zoom
    zoom_factor = random.uniform(0.7, 1.3)
    new_size = (int(image.shape[1] * zoom_factor), int(image.shape[0] * zoom_factor))
    image = cv2.resize(image, new_size)

    # Shear
    shear_factor = random.uniform(-0.2, 0.2)
    shear_matrix = np.array([[1, shear_factor, 0], [0, 1, 0]])
    image = cv2.warpAffine(image, shear_matrix, (image.shape[1], image.shape[0]))

    return image

In [608]:
# List semua kelas dalam direktori latihan
class_list = sorted(os.listdir(train_dir))

# Membuat kamus untuk menyimpan indeks kelas
class_indices = {class_name: i for i, class_name in enumerate(class_list)}

# Contoh penggunaan
print(class_indices)


{'diabetic_images': 0, 'normal_images': 1}


In [619]:
target_size = (150, 150)
batch_size = 2

# List semua file gambar dalam direktori latihan
file_list = [os.path.join(train_dir, filename) for filename in os.listdir(train_dir)]
# Fungsi untuk mengubah dan memuat batch gambar
def image_generator(file_list):
    while True:
        if len(file_list) < batch_size:
            raise ValueError("Batch size is larger than the number of available images.")
        batch_files = random.sample(file_list, batch_size)
        batch_images = []
        for file_path in batch_files:
            image = read_image(file_path, target_size)
            image = augment_image(image)
            batch_images.append(image)
        yield np.array(batch_images)

# Contoh penggunaan generator gambar
train_generator = image_generator(file_list)


In [620]:
validation_split = 0.1  # Persentase data validasi

# List semua file gambar dalam direktori latihan
file_list = [os.path.join(train_dir, filename) for filename in os.listdir(train_dir)]

# Mengacak urutan file gambar
random.shuffle(file_list)

# Menghitung jumlah data validasi berdasarkan validation_split
num_validation_samples = int(validation_split * len(file_list))

# Memisahkan data latihan dan validasi
train_files = file_list[:-num_validation_samples]
validation_files = file_list[-num_validation_samples:]

# Membuat instance generator data latihan
train_generator = image_generator(train_files)

# Membuat instance generator data validasi
validation_generator = image_generator(validation_files)

In [649]:
class Conv2D:
    def __init__(self, num_filters, kernel_size, activation):
        self.num_filters = num_filters
        self.kernel_size = kernel_size
        self.activation = activation
        
        # Initialize weights and biases
        self.weights = np.random.randn(num_filters, kernel_size, kernel_size)
        self.biases = np.zeros(num_filters)

    def initialize(self, input_shape):
        input_channels = input_shape[-1]
        kernel_height, kernel_width = self.kernel_size[0], self.kernel_size[1]
        self.weights = np.random.randn(kernel_height, kernel_width, input_channels, self.num_filters)
        self.bias = np.zeros(self.num_filters)

    def forward(self, input_data):
        input_height, input_width, _ = input_data.shape
        kernel_height, kernel_width = self.kernel_size[0], self.kernel_size[1]
        output_height = input_height - kernel_height + 1
        output_width = input_width - kernel_width + 1
        output = np.zeros((input_data.shape[0], output_height, output_width, self.num_filters))

        # Convolution operation
        for i in range(output_height):
            for j in range(output_width):
                for k in range(self.num_filters):
                    output[:, i, j, k] = np.sum(
                        input_data[:, i:i + kernel_height, j:j + kernel_width, :] * self.weights[:, :, :, k],
                        axis=(1, 2, 3)
                    ) + self.bias[k]

        if self.activation == "relu":
            output = np.maximum(output, 0)

        return output

In [622]:
class MaxPool2D:
    def __init__(self, pool_size, strides):
        self.pool_size = pool_size
        self.strides = strides

    def forward(self, input_data):
        input_height, input_width, num_channels = input_data.shape[1:]
        output_height = (input_height - self.pool_size[0]) // self.strides[0] + 1
        output_width = (input_width - self.pool_size[1]) // self.strides[1] + 1
        output = np.zeros((input_data.shape[0], output_height, output_width, num_channels))

        for i in range(output_height):
            for j in range(output_width):
                receptive_field = input_data[:, i*self.strides[0]:i*self.strides[0]+self.pool_size[0],
                                             j*self.strides[1]:j*self.strides[1]+self.pool_size[1], :]
                output[:, i, j, :] = np.amax(receptive_field, axis=(1, 2))

        return output

In [623]:
class Dense:
    def __init__(self, units, activation="relu"):
        self.units = units
        self.activation = activation
        self.weights = None
        self.bias = None

    def initialize(self, input_shape):
        input_size = np.prod(input_shape)
        self.weights = np.random.randn(input_size, self.units) / np.sqrt(input_size)
        self.bias = np.zeros(self.units)

    def forward(self, input_data):
        input_flattened = input_data.reshape((input_data.shape[0], -1))
        output = np.dot(input_flattened, self.weights) + self.bias

        if self.activation == "relu":
            output = np.maximum(output, 0)
        elif self.activation == "softmax":
            exp_output = np.exp(output)
            output = exp_output / np.sum(exp_output, axis=1, keepdims=True)

        return output
    def get_weights(self):
        return self.weights, self.bias

In [624]:
class Flatten:
    def forward(self, input_data):
        return input_data.reshape((input_data.shape[0], -1))
    
    

In [625]:
def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))  # Avoid numerical instability by subtracting the maximum value
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

In [626]:
# Membangun model
model = []
model.append(Conv2D(num_filters=32, kernel_size=3, activation="relu"))
model.append(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
model.append(Conv2D(num_filters=64, kernel_size=3, activation="relu"))
model.append(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
model.append(Conv2D(num_filters=128, kernel_size=3, activation="relu"))
model.append(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
model.append(Flatten())
model.append(Dense(units=256, activation="relu"))
model.append(Dense(units=256, activation="relu"))
# model.append(Dense(units=2, activation="softmax"))

In [627]:
# Cetak ringkasan model
for i, layer in enumerate(model):
    print(f"Layer {i + 1}")
    print("-------------")
    print(f"Type: {type(layer).__name__}")
    if isinstance(layer, Conv2D):
        print(f"Activation: {layer.activation}")
        print(f"Filters: {layer.num_filters}")
        print(f"Kernel Size: {layer.kernel_size}")
    elif isinstance(layer, Dense):
        print(f"Activation: {layer.activation}")
        print(f"Units: {layer.units}")
    print()

Layer 1
-------------
Type: Conv2D
Activation: relu
Filters: 32
Kernel Size: 3

Layer 2
-------------
Type: MaxPool2D

Layer 3
-------------
Type: Conv2D
Activation: relu
Filters: 64
Kernel Size: 3

Layer 4
-------------
Type: MaxPool2D

Layer 5
-------------
Type: Conv2D
Activation: relu
Filters: 128
Kernel Size: 3

Layer 6
-------------
Type: MaxPool2D

Layer 7
-------------
Type: Flatten

Layer 8
-------------
Type: Dense
Activation: relu
Units: 256

Layer 9
-------------
Type: Dense
Activation: relu
Units: 256



In [628]:
#model optimizer
class AdamaxOptimizer:
    def __init__(self, learning_rate=0.001, beta1=0.9, beta2=0.999):
        self.learning_rate = learning_rate
        self.beta1 = beta1
        self.beta2 = beta2
        self.m = None
        self.v = None
        self.t = 0

    def initialize(self, parameters):
        self.m = [np.zeros_like(param) for param in parameters]
        self.v = [np.zeros_like(param) for param in parameters]

    def update(self, parameters, gradients):
        self.t += 1

        for i in range(len(parameters)):
            self.m[i] = self.beta1 * self.m[i] + (1 - self.beta1) * gradients[i]
            self.v[i] = np.maximum(self.beta2 * self.v[i], np.abs(gradients[i]))
            lr_t = self.learning_rate / (1 - self.beta1 ** self.t)

            # Update weights
            parameters[i].weights -= lr_t * self.m[i] / (self.v[i] + 1e-8)

            # Update biases
            parameters[i].bias -= lr_t * np.mean(self.m[i]) / (np.mean(self.v[i]) + 1e-8)


In [629]:
#Membangun model optimizer
optimizer = AdamaxOptimizer(learning_rate=0.001)

In [630]:
# Inisialisasi bobot dan bias model
parameters = []
for layer in model:
    if isinstance(layer, Conv2D) or isinstance(layer, Dense):
        parameters.append(layer.weights)
        parameters.append(layer.bias)

optimizer.initialize(parameters)

In [631]:
# Definisikan loss function dan metrics
def categorical_crossentropy(y_true, y_pred):
    y_pred = np.clip(y_pred, 1e-7, 1 - 1e-7)  # Avoid numerical instability when taking the logarithm
    return -np.mean(y_true * np.log(y_pred))

def accuracy(y_true, y_pred):
    y_pred_binary = (y_pred > 0.5).astype(int)
    return np.mean(y_true == y_pred_binary)

In [632]:
# Fungsi untuk menghitung loss antara output model dan label batch
def calculate_loss(output, batch_y):
    # Konversi label batch menjadi array numpy
    batch_y = np.array(batch_y)

    # Hitung loss antara output dan label menggunakan metode yang sesuai
    loss = np.mean((output - batch_y)**2)

    return loss

In [633]:
def calculate_gradients(loss, weights, biases, train_data_flat, train_labels_binary):
    num_samples = train_data_flat.shape[0]
    
    # Compute y_pred
    logits = np.dot(train_data_flat, weights) + biases
    probabilities = softmax(logits)
    y_pred = np.argmax(probabilities, axis=1)
    
    # Compute gradient of loss with respect to logits
    gradient_logits = probabilities.copy()
    
    # Backpropagation
    gradient_logits[range(num_samples), train_labels_binary.flatten()] -= 1
    gradient_logits /= num_samples
    
    # Compute gradients of loss with respect to weights and biases
    d_weights = np.dot(train_data_flat.T, gradient_logits)
    d_biases = np.sum(gradient_logits, axis=0)
    
    gradients = {'weights': d_weights, 'biases': d_biases}
    
    return gradients


In [634]:
# Fungsi untuk memperbarui parameter model menggunakan gradien
def update_parameters(parameters, gradients, learning_rate):
    updated_parameters = {}

    for param_name, gradient in gradients.items():
        updated_parameters[param_name] = parameters[param_name] - learning_rate * gradient

    return updated_parameters



In [635]:
def train_model(train_data, train_labels, epochs, learning_rate):
    num_samples = train_data.shape[0]
    input_dim = train_data.shape[1:]
    num_classes = len(np.unique(train_labels))

    # Initialize weights and biases
    weights = np.random.randn(np.prod(input_dim), num_classes)
    biases = np.zeros(num_classes)

    # Flatten the input data
    train_data_flat = train_data.reshape(num_samples, -1)

    # Convert labels to binary format
    train_labels_binary = np.expand_dims(train_labels, axis=1)

    # Training loop
    for epoch in range(epochs):
        # Forward propagation
        logits = np.dot(train_data_flat, weights) + biases
        probabilities = softmax(logits)

        # Compute loss
        loss = categorical_crossentropy(train_labels_binary, probabilities)
        average_loss = np.mean(loss)
        
        # Backpropagation
        gradient = probabilities
        gradient[range(num_samples), train_labels] -= 1
        gradient /= num_samples

        d_weights = np.dot(train_data_flat.T, gradient)
        d_biases = np.sum(gradient, axis=0)

        # Update weights and biases
        weights -= learning_rate * d_weights
        biases -= learning_rate * d_biases
        # Print epoch information
        print(f"Epoch {epoch + 1}/{epochs} - Loss: {average_loss}")


    return weights


In [636]:
# Mendefinisikan folder yang berisi data latihan
train_folder = 'Data_Kurangi_Filter/Training/'

# Mendefinisikan kelas atau label yang sesuai dengan nama folder
labels = ['normal_images', 'diabetic_images']

# Menginisialisasi list untuk menyimpan data latihan dan label
train_data = []
train_labels = []

# Melakukan iterasi pada setiap folder kelas
for label_index, label in enumerate(labels):
    folder_path = os.path.join(train_folder, label)
    
    # Melakukan iterasi pada setiap file dalam folder kelas
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        
        # Membaca dan memproses gambar
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Ubah ke format warna RGB jika perlu
        image = cv2.resize(image, (224, 224))  # Ubah ukuran gambar jika perlu
        image = image / 255.0  # Normalisasi pixel
        image
        # Menyimpan data latihan dan label ke dalam list
        train_data.append(image)
        train_labels.append(label_index)

# Mengubah list menjadi array numpy
train_data = np.array(train_data)
train_labels = np.array(train_labels)
train_data = np.expand_dims(train_data, axis=-1)

learning_rate = 0.01
epochs = 150

weights = train_model(train_data, train_labels, epochs, learning_rate)


Epoch 1/150 - Loss: 4.020898737730378
Epoch 2/150 - Loss: 3.854163526985601
Epoch 3/150 - Loss: 3.9839916201943115
Epoch 4/150 - Loss: 3.9442058143423555
Epoch 5/150 - Loss: 3.896378118485869
Epoch 6/150 - Loss: 3.8572704167122644
Epoch 7/150 - Loss: 3.7619048910782267
Epoch 8/150 - Loss: 3.7493612610128806
Epoch 9/150 - Loss: 3.7014774973378373
Epoch 10/150 - Loss: 3.7766137370484376
Epoch 11/150 - Loss: 3.9376762573726514
Epoch 12/150 - Loss: 3.9967137584596726
Epoch 13/150 - Loss: 3.9995370600724196
Epoch 14/150 - Loss: 3.9972628313128804
Epoch 15/150 - Loss: 3.9989802643887957
Epoch 16/150 - Loss: 3.996625761098409
Epoch 17/150 - Loss: 3.9888470028396386
Epoch 18/150 - Loss: 3.9845168662843777
Epoch 19/150 - Loss: 3.9811320559607144
Epoch 20/150 - Loss: 3.9420699955913303
Epoch 21/150 - Loss: 3.9805281807474744
Epoch 22/150 - Loss: 3.905214080291628
Epoch 23/150 - Loss: 3.9879667322668184
Epoch 24/150 - Loss: 3.897695517652053
Epoch 25/150 - Loss: 4.0014014232451425
Epoch 26/150 - 

In [654]:
# Flatten hasil segmentasi menjadi vektor input
input_nn = train_data[1].flatten()
# Normalisasi input menjadi nilai antara 0 dan 1
input_nn = input_nn / np.max(input_nn)

In [655]:
# Fungsi aktivasi sigmoid
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Fungsi aktivasi turunan sigmoid
def sigmoid_derivative(x):
    return x * (1 - x)

# Kelas NeuralNetwork
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        # Inisialisasi bobot dengan nilai acak
        self.weights1 = np.random.randn(input_size, hidden_size)
        self.weights2 = np.random.randn(hidden_size, output_size)

    def forward(self, X):
        # Perhitungan output dari input melalui hidden layer
        self.hidden_output = sigmoid(np.dot(X, self.weights1))

        # Perhitungan output akhir dari hidden layer melalui output layer
        self.output = sigmoid(np.dot(self.hidden_output, self.weights2))

    def train(self, X, y, epochs):
        for epoch in range(epochs):
            # Feedforward
            self.forward(X)

            # Perhitungan error
            error = y - self.output

            # Perhitungan gradien dan penyesuaian bobot
            delta_output = error * sigmoid_derivative(self.output)
            delta_hidden = np.dot(delta_output, self.weights2.T) * sigmoid_derivative(self.hidden_output)

            self.weights2 += np.dot(self.hidden_output.T, delta_output)
            self.weights1 += np.dot(X.T, delta_hidden)

In [656]:
# Inisialisasi dan melatih neural network
nn = NeuralNetwork(input_size=input_nn.shape[0], hidden_size=4, output_size=1)
nn.train(np.expand_dims(input_nn, axis=0), np.array([[1]]), epochs=10000)

In [657]:
prediksi = nn.output
print(prediksi)

[[0.99588483]]


In [638]:
def plot_model():
    input_layer = 'Input Layer:      [Input Shape]'
    hidden_layer = 'Hidden Layer: [Hidden Shape]'
    output_layer = 'Output Layer:   [Output Shape]'

    # Menentukan panjang garis horizontal sesuai dengan panjang layer terpanjang
    max_length = max(len(input_layer), len(hidden_layer), len(output_layer))
    horizontal_line = '-' * max_length

    # Menampilkan plot model
    print(input_layer)
    print(horizontal_line)
    print(hidden_layer)
    print(horizontal_line)
    print(output_layer)

# Memanggil fungsi plot_model untuk membuat plot model
plot_model()


Input Layer:      [Input Shape]
-------------------------------
Hidden Layer: [Hidden Shape]
-------------------------------
Output Layer:   [Output Shape]


In [643]:
import cv2
import numpy as np

# Define the necessary functions for preprocessing and prediction
def preprocess_image(image_path, target_size):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read the image in grayscale
    img = cv2.resize(img, target_size)
    img = cv2.addWeighted(img, 4, cv2.GaussianBlur(img, (0, 0), 40), -4, 128)
    img = img.astype('float32') / 255.0  # Normalize the image
    img = np.expand_dims(img, axis=0)  # Add an extra dimension
    return img

def predict(image_array, model):
    prediction = model.predict(image_array)
    predicted_class = np.argmax(prediction)  # Retrieve the predicted class index
    confidence = prediction[0, predicted_class]  # Retrieve the confidence score of the predicted class
    return predicted_class, confidence

# Set the path to the input image
pred_path = "normal_images/10_left.jpeg"
target_size = (150, 150)

# Preprocess the image
preprocessed_image = preprocess_image(pred_path, target_size)

# Make the prediction using your custom model
predicted_class, confidence = predict(preprocessed_image, model)

# Process and display the prediction results
class_names = ['normal_images', 'diabetic_images']  # Define the list of class names
predicted_class_name = class_names[predicted_class]
print("The image is predicted to belong to the category {} with {:.2f}% confidence."
      .format(predicted_class_name, confidence * 100))


AttributeError: 'list' object has no attribute 'predict'

In [ ]:
def convolution_operation(image_array, num_filters, kernel_size, activation):
    # Get the dimensions of the image
    image_height, image_width, _ = image_array.shape

    # Extract the height and width values from the kernel_size tuple
    kernel_height, kernel_width = kernel_size

    # Calculate the padding size based on the kernel size
    padding_height = kernel_height // 2
    padding_width = kernel_width // 2

    # Add padding to the image
    padded_image = np.pad(image_array, ((padding_height, padding_height), (padding_width, padding_width), (0, 0)), mode='constant')

    # Initialize the output result
    output_height = image_height
    output_width = image_width
    output = np.zeros((output_height, output_width, num_filters))

    # Apply convolution operation
    for i in range(output_height):
        for j in range(output_width):
            for k in range(num_filters):
                receptive_field = padded_image[i:i + kernel_height, j:j + kernel_width, :]
                num_channels = receptive_field.shape[-1]
                filter_weights = np.random.randn(kernel_height, kernel_width, num_channels)
                output[i, j, k] = np.sum(receptive_field * filter_weights)

    # Apply activation function
    if activation == "relu":
        output = np.maximum(output, 0)
    elif activation == "sigmoid":
        output = 1 / (1 + np.exp(-output))

    return output


In [ ]:
def flatten_operation(image_array):
    # Flatten the image array into a 1D array
    flattened_array = image_array.flatten()

    return flattened_array


In [ ]:
import numpy as np

def dense_layer(input_data, weights, activation):
    # Perform matrix multiplication between input_data and weights
    output = np.dot(input_data, weights)
    
    if activation == 'relu':
        output = relu(output)
    elif activation == 'sigmoid':
        output = sigmoid(output)
    
    return output

def relu(x):
    return np.maximum(0, x)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))


In [ ]:
def predict(image_array, model):
    result = image_array.copy()
    for layer in model:
        if isinstance(layer, Conv2D):
            num_filters = layer.num_filters
            kernel_size = layer.kernel_size
            activation = layer.activation

            # Perform convolution operation manually
            result = convolution_operation(result, num_filters, kernel_size, activation)
        elif isinstance(layer, Flatten):
            # Perform flattening operation
            result = flatten_operation(result)
        elif isinstance(layer, Dense):
            weights = layer.weights
            activation = layer.activation

            # Perform dense operation manually
            result = dense_layer(result, weights, activation)

    predicted_class = np.argmax(result)  # Retrieve the predicted class index
    return predicted_class
